## insect modeling

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, cross_val_predict, RandomizedSearchCV

# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.metrics import classification_report

def split_train_test(X,y, test_ratio):
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio, random_state=1)
    for train_idx, test_idx in split.split(X, y):
        X_train = X.loc[train_idx]
        X_test = X.loc[test_idx]
        y_train = y.loc[train_idx]
        y_test = y.loc[test_idx]
    return X_train, X_test, y_train, y_test

### RF 2% ~ 5%
* with randomized grid search
* adjusting class_weight
* save models with roc_auc_score over 0.7

In [4]:
def rf_gridSearch_train(X_train, y_train, save, file_dir=None):
    param_grid = {'bootstrap': [True, False],
     'max_depth': [ 80, 90, 100, None],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [600, 800, 1000]
                 }

    # Create a based model
    rf = RandomForestClassifier()
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                              cv = 3, n_jobs = -1, verbose = 2)

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # best params

    rf_best_est = grid_search.best_estimator_
    rf = rf_best_est.fit(X_train, y_train)

    rf.fit(X_train, y_train)
    
    if save:
        pickle.dump(rf, open(file_dir, 'wb'))
    
    return rf

def print_result(model, X_test, y_test):
    pred = model.predict(X_test)

    print(classification_report(y_test, pred))
    
    print('ROC/AUC: ', roc_auc_score(y_test, pred))
    print('ACC: ', accuracy_score(y_test, pred))
    print('PREC: ', precision_score(y_test, pred))
    print('RECALL: ', recall_score(y_test, pred))
    print('F1: ', f1_score(y_test, pred))

    print('Confusion Matrix: \n', confusion_matrix(y_test, pred))

In [5]:
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/2%~5%/'
dir_list = os.listdir(dir_name)
dict_model_rf = {}
save_dir = '/Users/shinbo/Desktop/contest/environ/model/insect/'


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}




for _dir in dir_list:
    
    tmp = pd.read_csv(dir_name + _dir)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad']]
    y = tmp['y']
    
    weight = len(np.where(y == 0)[0]) / len(np.where(y == 1)[0])

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)
    
    # Create a based model
    rf = RandomForestClassifier(class_weight={0:1,1:weight})
    # Instantiate the grid search model
    grid_search = RandomizedSearchCV(estimator = rf, param_distributions = params, 
                                     cv = 3, verbose=2, random_state=42, n_jobs = -1)

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # best params

    rf_best_est = grid_search.best_estimator_
    rf = rf_best_est.fit(X_train, y_train)

    rf.fit(X_train, y_train)
   
    
    pred = rf.predict(X_test)
    roc_score = roc_auc_score(y_test,pred)
    if roc_score >= 0.7:        
        print('\n')
        print(f'########################### classification report for {_dir[:-4]} ###########################')
        print_result(rf,X_test,y_test)
        dict_model_rf[_dir[:-4]] = rf
        print('\n')

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.3s finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.7s finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    9.3s finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.9s finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    9.2s finished


### RF 5%~
* with randomized grid search
* adjusting class_weight
* save models with roc_auc_score over 0.7

In [22]:
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/5%~/'
dir_list = os.listdir(dir_name)
save_dir = '/Users/shinbo/Desktop/contest/environ/model/insect/'


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


for _dir in dir_list:
    
    tmp = pd.read_csv(dir_name + _dir)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad']]
    y = tmp['y']
    
    weight = len(np.where(y == 0)[0]) / len(np.where(y == 1)[0])

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)
    
    # Create a based model
    rf = RandomForestClassifier(class_weight={0:1,1:weight})
    # Instantiate the grid search model
    grid_search = RandomizedSearchCV(estimator = rf, param_distributions = params, 
                                     cv = 3, verbose=2, random_state=42, n_jobs = -1)

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # best params

    rf_best_est = grid_search.best_estimator_
    rf = rf_best_est.fit(X_train, y_train)

    rf.fit(X_train, y_train)
   
    
    pred = rf.predict(X_test)
    roc_score = roc_auc_score(y_test,pred)
    if roc_score >= 0.7:        
        print('\n')
        print(f'########################### classification report for {_dir[:-4]} ###########################')
        print_result(rf,X_test,y_test)
        dict_model_rf[_dir[:-4]] = rf
        print('\n')

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   15.4s finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.0s finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


...

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.4s finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.7s finished


KeyError: "None of [Index(['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9',\n       'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17',\n       'bio18', 'bio19', 'pet', 'rh', 'srad'],\n      dtype='object')] are in the [columns]"

In [ ]:
pickle.dump(dict_model_rf, open('../model_dict_weight_adj/insect_rf.pickle', 'wb'))

### xgb 2% ~ 5%
* with randomized grid search
* adjusting scale_pos_weight
* save models with roc_auc_score over 0.7

In [16]:
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/2%~5%/'
dir_list = os.listdir(dir_name)
dict_model_xgb = {}
save_dir = '/Users/shinbo/Desktop/contest/environ/model/insect/'
for _dir in dir_list:
    if _dir[:-4] not in name_list:
        continue
    tmp = pd.read_csv(dir_name + _dir)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad']]
    y = tmp['y']
    
    weight = len(np.where(y == 0)[0]) / len(np.where(y == 1)[0])

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)

        # Create XGB Classifier object
    xgb_clf = XGBClassifier(scale_pos_weight = weight)

    # Create parameter grid
    parameters = {"learning_rate": [0.1, 0.01, 0.001],
                   "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
                   "max_depth": [2, 4, 7, 10],
                   "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
                   "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
                   "reg_alpha": [0, 0.5, 1],
                   "reg_lambda": [1, 1.5, 2, 3, 4.5],
                   "min_child_weight": [1, 3, 5, 7],
                   "n_estimators": [100, 250, 500, 1000]}

    print(f'starting optimizing for {_dir[:-4]}')
    # Create RandomizedSearchCV Object
    xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = "roc_auc",
                                 cv = 3, random_state = 40 , n_jobs = -1, verbose = 2)
    print('done tuning')

    # Fit the model
    model_xgboost = xgb_rscv.fit(X_train, y_train)
    
    xgb_best = model_xgboost.best_estimator_
    
    final_xgb = xgb_best.fit(X_train, y_train)
    
   
    
    pred = final_xgb.predict(X_test)
    roc_score = roc_auc_score(y_test,pred)
    if roc_score >= 0.7:        
        print('\n')
        print(f'########################### classification report for {_dir[:-4]} ###########################')
        print_result(final_xgb,X_test,y_test)
        dict_model_xgb[_dir[:-4]] = final_xgb
        print('\n')

starting optimizing for 검거세미밤나방
done tuning
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.




########################### classification report for 검거세미밤나방 ###########################
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       229
           1       0.22      0.50      0.31         8

    accuracy                           0.92       237
   macro avg       0.60      0.72      0.63       237
weighted avg       0.96      0.92      0.94       237

ROC/AUC:  0.7194323144104803
ACC:  0.9240506329113924
PREC:  0.2222222222222222
RECALL:  0.5
F1:  0.30769230769230765
Confusion Matrix: 
 [[215  14]
 [  4   4]]


starting optimizing for 톱다리개미허리노린재
done tuning
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


starting optimizing for 솔잎말이나방
done tuning
Fitting 3 folds for each of 10 candidates, totalling 30 fits


...

[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.




########################### classification report for 남방제비나비 ###########################
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       227
           1       0.32      0.70      0.44        10

    accuracy                           0.92       237
   macro avg       0.65      0.82      0.70       237
weighted avg       0.96      0.92      0.94       237

ROC/AUC:  0.8169603524229075
ACC:  0.9240506329113924
PREC:  0.3181818181818182
RECALL:  0.7
F1:  0.4375
Confusion Matrix: 
 [[212  15]
 [  3   7]]


starting optimizing for 목화진딧물
done tuning
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.




########################### classification report for 목화진딧물 ###########################
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       232
           1       0.25      0.60      0.35         5

    accuracy                           0.95       237
   macro avg       0.62      0.78      0.66       237
weighted avg       0.98      0.95      0.96       237

ROC/AUC:  0.780603448275862
ACC:  0.9535864978902954
PREC:  0.25
RECALL:  0.6
F1:  0.35294117647058826
Confusion Matrix: 
 [[223   9]
 [  2   3]]


starting optimizing for 꼬마광채꽃벌
done tuning
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished


### xgb 5%~
* with randomized grid search
* adjusting scale_pos_weight
* save models with roc_auc_score over 0.7

In [17]:
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/5%~/'
dir_list = sorted(os.listdir(dir_name))[1:]
save_dir = '/Users/shinbo/Desktop/contest/environ/model/insect/'
for _dir in dir_list:
    if _dir[:-4] not in name_list:
        continue
    tmp = pd.read_csv(dir_name + _dir)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad']]
    y = tmp['y']
    
    weight = len(np.where(y == 0)[0]) / len(np.where(y == 1)[0])

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)

        # Create XGB Classifier object
    xgb_clf = XGBClassifier(scale_pos_weight = weight)

    # Create parameter grid
    parameters = {"learning_rate": [0.1, 0.01, 0.001],
                   "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
                   "max_depth": [2, 4, 7, 10],
                   "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
                   "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
                   "reg_alpha": [0, 0.5, 1],
                   "reg_lambda": [1, 1.5, 2, 3, 4.5],
                   "min_child_weight": [1, 3, 5, 7],
                   "n_estimators": [100, 250, 500, 1000]}

    print(f'starting optimizing for {_dir[:-4]}')
    # Create RandomizedSearchCV Object
    xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = "roc_auc",
                                 cv = 5, random_state = 40 , n_jobs = -1, verbose = 2)
    print('done tuning')

    # Fit the model
    model_xgboost = xgb_rscv.fit(X_train, y_train)
    
    xgb_best = model_xgboost.best_estimator_
    
    final_xgb = xgb_best.fit(X_train, y_train)
    
   
    
    pred = final_xgb.predict(X_test)
    roc_score = roc_auc_score(y_test,pred)
    if roc_score >= 0.7:        
        print('\n')
        print(f'########################### classification report for {_dir[:-4]} ###########################')
        print_result(final_xgb,X_test,y_test)
        dict_model_xgb[_dir[:-4]] = final_xgb
        print('\n')
    else:
        print('\n')
        print(f'########################### classification report for {_dir[:-4]} ###########################')
        print(f'아쉽게도 {_dir[:-4]}은 우리와 함께가지 못할듯 ㅎㅎ')
        print('\n')

starting optimizing for 검은다리실베짱이
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.




########################### classification report for 검은다리실베짱이 ###########################
아쉽게도 검은다리실베짱이은 우리와 함께가지 못할듯 ㅎㅎ


starting optimizing for 게눈노린재
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.2s finished




########################### classification report for 게눈노린재 ###########################
아쉽게도 게눈노린재은 우리와 함께가지 못할듯 ㅎㅎ


starting optimizing for 극동혹개미
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


...



########################### classification report for 운계방패멸구 ###########################
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       225
           1       0.25      0.58      0.35        12

    accuracy                           0.89       237
   macro avg       0.61      0.74      0.65       237
weighted avg       0.94      0.89      0.91       237

ROC/AUC:  0.7450000000000001
ACC:  0.890295358649789
PREC:  0.25
RECALL:  0.5833333333333334
F1:  0.35000000000000003
Confusion Matrix: 
 [[204  21]
 [  5   7]]


starting optimizing for 줄점팔랑나비
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.




########################### classification report for 줄점팔랑나비 ###########################
              precision    recall  f1-score   support

           0       0.89      0.73      0.80       185
           1       0.42      0.69      0.52        52

    accuracy                           0.72       237
   macro avg       0.66      0.71      0.66       237
weighted avg       0.79      0.72      0.74       237

ROC/AUC:  0.711018711018711
ACC:  0.7215189873417721
PREC:  0.4186046511627907
RECALL:  0.6923076923076923
F1:  0.5217391304347826
Confusion Matrix: 
 [[135  50]
 [ 16  36]]


starting optimizing for 청띠제비나비
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.9s finished




########################### classification report for 청띠제비나비 ###########################
              precision    recall  f1-score   support

           0       0.99      0.92      0.95       225
           1       0.36      0.83      0.50        12

    accuracy                           0.92       237
   macro avg       0.67      0.88      0.73       237
weighted avg       0.96      0.92      0.93       237

ROC/AUC:  0.8766666666666668
ACC:  0.9156118143459916
PREC:  0.35714285714285715
RECALL:  0.8333333333333334
F1:  0.5
Confusion Matrix: 
 [[207  18]
 [  2  10]]




In [18]:
pickle.dump(dict_model_xgb, open('../model_dict_weight_adj/insect_xgb_exclude.pickle', 'wb'))

### data saving

In [19]:
# etc 2% ~ 5%
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/2%~5%/'
dir_list = os.listdir(dir_name)

insect_data_dict = {}
for _dir in dir_list:
    
    if _dir[:-4] in list(dict_model_xgb.keys()):
        tmp = pd.read_csv('/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/2%~5%/' + _dir)
        X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
               'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
               'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad']]
        y = tmp['y']

        X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)
        insect_data_dict[_dir[:-4]] = {'X_train':X_train, 'X_test':X_test, 'y_train':y_train, 'y_test':y_test}
        
        
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/5%~/'
dir_list = os.listdir(dir_name)

for _dir in dir_list:
    
    if _dir[:-4] in list(dict_model_xgb.keys()):
        tmp = pd.read_csv('/Users/shinbo/Desktop/contest/environ/concat_data/insect/etc/5%~/' + _dir)
        X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
               'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
               'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad']]
        y = tmp['y']

        X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)
        insect_data_dict[_dir[:-4]] = {'X_train':X_train, 'X_test':X_test, 'y_train':y_train, 'y_test':y_test}

In [20]:
pickle.dump(insect_data_dict, open('../model_dict_weight_adj/insect_data.pickle','wb'))